## 8.1 分层索引
    分层索引是pandas的重要特性，允许在轴向上拥有多个(两个或两个以上)索引层级
    分层索引提供了一种在更低维度的形式中处理更高维度数据的方式

In [8]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9),
                 index = [list('aaabbccdd'), 
                         [1, 2, 3, 1, 3, 1, 2, 2, 3]]) # 返回一个以MultiIndex作为索引的Series对象
print(data)
print(data.index)

a  1    0.404044
   2    0.705326
   3    2.287387
b  1    0.394717
   3   -0.679748
c  1    1.073946
   2    0.104329
d  2    0.025561
   3    1.325090
dtype: float64
MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])


In [9]:
# 通过分层索引对象，可以简洁地选择出数据的子集
data['b']

1    0.394717
3   -0.679748
dtype: float64

In [10]:
data['b':'c']

b  1    0.394717
   3   -0.679748
c  1    1.073946
   2    0.104329
dtype: float64

In [12]:
data.loc[['b', 'd']]

b  1    0.394717
   3   -0.679748
d  2    0.025561
   3    1.325090
dtype: float64

In [13]:
data.loc[:, 2] # 在内部层级中进行选择也是可以的

a    0.705326
c    0.104329
d    0.025561
dtype: float64

In [14]:
# 分层索引在重塑数据透视表等分组操作中扮演了重要角色
data.unstack()

,1,2,3
a,0.404044,0.705326,2.287387
b,0.394717,NaN,-0.679748
c,1.073946,0.104329,NaN
d,NaN,0.025561,1.325090


In [15]:
data.unstack().stack()

a  1    0.404044
   2    0.705326
   3    2.287387
b  1    0.394717
   3   -0.679748
c  1    1.073946
   2    0.104329
d  2    0.025561
   3    1.325090
dtype: float64

In [27]:
# 在DataFrame中，每个轴都可以有分层索引
frame = pd.DataFrame(np.arange(12).reshape(4,3),
                     index = [['a', 'a', 'b', 'b'], 
                              [1, 2, 1, 2]],
                     columns = [['Ohio','Ohio', 'Colorado'], 
                                ['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['State', 'Color']

frame

State      Ohio     Colorado
Color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [23]:
frame['Ohio']

Green  Red
a 1      0    1
  2      3    4
b 1      6    7
  2      9   10

In [24]:
# MultiIndex对象可以使用构造函数创建并复用
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['State', 'Color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['State', 'Color'])

### 5.1.1 重排序和层级排序

In [28]:
# 重新排列轴上的层级顺序，或按照特定层级的值对数据进行排序
# swaplevel接收两个层级序号或层级名称，返回一个层级改变的新对象

frame.swaplevel('key1', 'key2')

State      Ohio     Colorado
Color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [31]:
# sort_index只能在单一层级上对数据进行排序
frame

State      Ohio     Colorado
Color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [32]:
frame.sort_index(level=1)

State      Ohio     Colorado
Color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [33]:
frame.swaplevel(0,1).sort_index(level=0)

State      Ohio     Colorado
Color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 5.1.2 按层级进行汇总统计

In [35]:
# DataFrame和Series中很多描述性和汇总性统计有一个level选项
# 通过level选项可以指定在某个特定轴上进行聚合

frame

State      Ohio     Colorado
Color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [36]:
frame.sum(level='key2')

State  Ohio     Colorado
Color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [37]:
frame.sum(level=0)

State  Ohio     Colorado
Color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

### 5.1.3 使用DataFrame的列进行索引

In [38]:
frame = pd.DataFrame({'a':range(7),
                      'b':range(7, 0, -1),
                      'c':['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd':[0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [40]:
# 使用set_index函数生成一个使用一个或多个列作为索引的新的DataFrame对象
# 默认情况下这些列会从DataFrame中移除
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [42]:
# 可以向set_index传入drop=False，对这些列进行保留
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [43]:
# reset_index是set_index的反操作，分层索引的索引层级会被移动到列中
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 5.2 联合与合并数据

In [44]:
# pandas可以通过多种方式将数据连接在一起
# pandas.merge根据一个或多个键将行进行连接
# pandas.concat使对象在轴向上进行联合或堆叠
# combine_first 实例方法允许将重叠的数据拼接在一起，以使用一个对象中的值填充另一个对象中的缺失值

In [45]:
### 5.2.1 数据库风格的DataFrame连接